In [1]:

import glob
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, average_precision_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import VotingClassifier
import matplotlib.pyplot as plt
import glob
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, average_precision_score
import matplotlib.pyplot as plt


In [2]:

monday_path = r"C:\Users\SREENITHI\fed_learning_cicids2017\MachineLearningCSV\MachineLearningCVE\Monday-WorkingHours.pcap_ISCX.csv"
tuesday_path = r"C:\Users\SREENITHI\fed_learning_cicids2017\MachineLearningCSV\MachineLearningCVE\Tuesday-WorkingHours.pcap_ISCX.csv"
wednesday_path = r"C:\Users\SREENITHI\fed_learning_cicids2017\MachineLearningCSV\MachineLearningCVE\Wednesday-workingHours.pcap_ISCX.csv"
thursday_m_path = r"C:\Users\SREENITHI\fed_learning_cicids2017\MachineLearningCSV\MachineLearningCVE\Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv"
thursday_a_path = r"C:\Users\SREENITHI\fed_learning_cicids2017\MachineLearningCSV\MachineLearningCVE\Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv"
friday_dos_path = r"C:\Users\SREENITHI\fed_learning_cicids2017\MachineLearningCSV\MachineLearningCVE\Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv"
friday_portscan_path = r"C:\Users\SREENITHI\fed_learning_cicids2017\MachineLearningCSV\MachineLearningCVE\Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv"
friday_morning_path = r"C:\Users\SREENITHI\fed_learning_cicids2017\MachineLearningCSV\MachineLearningCVE\Friday-WorkingHours-Morning.pcap_ISCX.csv"

monday = pd.read_csv(monday_path)
tuesday = pd.read_csv(tuesday_path)
wednesday = pd.read_csv(wednesday_path)
thursday_m = pd.read_csv(thursday_m_path)
thursday_a = pd.read_csv(thursday_a_path)
friday_dos = pd.read_csv(friday_dos_path)
friday_portscan = pd.read_csv(friday_portscan_path)
friday_morning = pd.read_csv(friday_morning_path)
data = pd.concat([monday, tuesday, wednesday, thursday_m, thursday_a, 
                  friday_dos, friday_portscan, friday_morning], ignore_index=True)

print("Dataset shape:", data.shape)
rows, cols = data.shape
print(f'Number of rows: {rows}')
print(f'Number of columns: {cols}')
print(f'Total cells: {rows * cols}')
data.columns

Dataset shape: (2830743, 79)
Number of rows: 2830743
Number of columns: 79
Total cells: 223628697


Index([' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', '

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2830743 entries, 0 to 2830742
Data columns (total 79 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0    Destination Port             int64  
 1    Flow Duration                int64  
 2    Total Fwd Packets            int64  
 3    Total Backward Packets       int64  
 4   Total Length of Fwd Packets   int64  
 5    Total Length of Bwd Packets  int64  
 6    Fwd Packet Length Max        int64  
 7    Fwd Packet Length Min        int64  
 8    Fwd Packet Length Mean       float64
 9    Fwd Packet Length Std        float64
 10  Bwd Packet Length Max         int64  
 11   Bwd Packet Length Min        int64  
 12   Bwd Packet Length Mean       float64
 13   Bwd Packet Length Std        float64
 14  Flow Bytes/s                  float64
 15   Flow Packets/s               float64
 16   Flow IAT Mean                float64
 17   Flow IAT Std                 float64
 18   Flow IAT Max         

In [4]:
dups = data[data.duplicated()]
print(f'Number of duplicates: {len(dups)}')

Number of duplicates: 308381


In [5]:
data.drop_duplicates(inplace = True)
data.shape

(2522362, 79)

In [6]:
missing_val = data.isna().sum()
print(missing_val.loc[missing_val > 0])

Flow Bytes/s    353
dtype: int64


In [7]:
numeric_cols = data.select_dtypes(include = np.number).columns
inf_count = np.isinf(data[numeric_cols]).sum()
print(inf_count[inf_count > 0])

# Replacing any infinite values (positive or negative) with NaN (not a number)
print(f'Initial missing values: {data.isna().sum().sum()}')

data.replace([np.inf, -np.inf], np.nan, inplace = True)

print(f'Missing values after processing infinite values: {data.isna().sum().sum()}')

Flow Bytes/s       1211
 Flow Packets/s    1564
dtype: int64
Initial missing values: 353
Missing values after processing infinite values: 3128


In [8]:
missing = data.isna().sum()
print(missing.loc[missing > 0])

Flow Bytes/s       1564
 Flow Packets/s    1564
dtype: int64


In [9]:
# Calculating missing value percentage in the dataset
mis_per = (missing / len(data)) * 100
mis_table = pd.concat([missing, mis_per.round(2)], axis = 1)
mis_table = mis_table.rename(columns = {0 : 'Missing Values', 1 : 'Percentage of Total Values'})

print(mis_table.loc[mis_per > 0])

                 Missing Values  Percentage of Total Values
Flow Bytes/s               1564                        0.06
 Flow Packets/s            1564                        0.06


In [13]:
med_flow_bytes = data['Flow Bytes/s'].median()
med_flow_packets = data['Flow Packets/s'].median()

print('Median of Flow Bytes/s: ', med_flow_bytes)
print('Median of Flow Packets/s: ', med_flow_packets)

# Filling missing values with median
data['Flow Bytes/s'].fillna(med_flow_bytes, inplace = True)
data['Flow Packets/s'].fillna(med_flow_packets, inplace = True)

Median of Flow Bytes/s:  3715.0378579999997
Median of Flow Packets/s:  69.742244285


In [15]:
# Filling missing values with median
data['Flow Bytes/s'].fillna(med_flow_bytes, inplace = True)
data['Flow Packets/s'].fillna(med_flow_packets, inplace = True)

In [16]:
print('Number of \'Flow Bytes/s\' missing values:', data['Flow Bytes/s'].isna().sum())
print('Number of \'Flow Packets/s\' missing values:', data['Flow Packets/s'].isna().sum())

Number of 'Flow Bytes/s' missing values: 0
Number of 'Flow Packets/s' missing values: 0


In [17]:
# Dropping columns with only one unique value
num_unique = data.nunique()
one_variable = num_unique[num_unique == 1]
not_one_variable = num_unique[num_unique > 1].index

dropped_cols = one_variable.index
data = data[not_one_variable]

print('Dropped columns:')
dropped_cols

Dropped columns:


Index(['Bwd PSH Flags', 'Bwd URG Flags', 'Fwd Avg Bytes/Bulk',
       'Fwd Avg Packets/Bulk', 'Fwd Avg Bulk Rate', 'Bwd Avg Bytes/Bulk',
       'Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate'],
      dtype='object')

In [18]:
# Dropping columns with only one unique value
num_unique = data.nunique()
one_variable = num_unique[num_unique == 1]
not_one_variable = num_unique[num_unique > 1].index

dropped_cols = one_variable.index
data = data[not_one_variable]

print('Dropped columns:')
dropped_cols

Dropped columns:


Index([], dtype='object')

In [24]:
from sklearn.preprocessing import StandardScaler

features = data.drop('Label', axis=1)
attacks = data['Label']

# Standardize features
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

print("Features shape:", features_scaled.shape)
print("Attacks shape:", attacks.shape)
print("Unique labels:", attacks.unique())


Features shape: (2522362, 70)
Attacks shape: (2522362,)
Unique labels: ['BENIGN' 'FTP-Patator' 'SSH-Patator' 'DoS slowloris' 'DoS Slowhttptest'
 'DoS Hulk' 'DoS GoldenEye' 'Heartbleed' 'Web Attack � Brute Force'
 'Web Attack � XSS' 'Web Attack � Sql Injection' 'Infiltration' 'DDoS'
 'PortScan' 'Bot']


In [25]:
from sklearn.decomposition import IncrementalPCA

size = len(features.columns) // 2
ipca = IncrementalPCA(n_components = size, batch_size = 500)
for batch in np.array_split(scaled_features, len(features) // 500):
    ipca.partial_fit(batch)

print(f'information retained: {sum(ipca.explained_variance_ratio_):.2%}')

information retained: 99.31%


In [26]:
transformed_features = ipca.transform(scaled_features)
new_data = pd.DataFrame(transformed_features, columns = [f'PC{i+1}' for i in range(size)])
new_data['Attack Type'] = attacks.values

In [27]:
new_data

,PC1,PC2,PC3,PC4,PC5,PC6,PC7,PC8,PC9,PC10,...,PC27,PC28,PC29,PC30,PC31,PC32,PC33,PC34,PC35,Attack Type
0,-2.357605,-0.054980,0.577353,0.734456,3.730369,0.234611,-0.015694,-0.215511,-0.278414,1.087306,...,-0.147827,-0.333229,1.488574,-0.161194,-0.220094,-0.004678,0.002594,0.033000,-0.014335,BENIGN
1,-2.883852,-0.069937,0.911290,1.763144,8.846136,0.620333,-0.056858,1.106734,1.906381,-2.756490,...,0.280804,-2.345977,2.240673,-0.637403,-0.372210,0.010427,0.000639,0.127035,-0.189619,BENIGN
2,-2.416927,-0.056683,0.615319,0.850871,4.304212,0.275732,-0.020437,-0.070440,-0.035232,0.664190,...,-0.101782,-0.544394,1.572942,-0.211371,-0.235800,-0.003149,0.002387,0.043104,-0.033774,BENIGN
3,-2.884703,-0.069978,0.912152,1.765260,8.851561,0.618596,-0.057028,1.104890,1.906808,-2.752517,...,0.279224,-2.333504,2.241475,-0.634668,-0.371014,0.010278,0.000649,0.126691,-0.189582,BENIGN
4,-1.505246,0.080887,-0.504399,0.290388,-0.539260,0.746153,0.100451,0.732386,-1.145858,-0.563193,...,-0.598816,0.116562,0.794736,-0.207833,0.038410,-0.014006,0.000994,-0.047731,-0.003299,BENIGN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2522357,-1.967967,-0.033741,0.213324,-0.626711,-1.005926,-0.604078,-0.053547,-0.512359,0.659115,-0.495693,...,-0.208086,0.044167,0.379626,-0.208258,-0.010394,-0.003753,0.000713,-0.022263,0.059741,BENIGN
2522358,-1.954698,-0.033582,0.193326,-0.720497,-1.224944,-0.852049,-0.072429,-0.599180,0.965421,-0.574084,...,-0.091746,0.197137,0.476501,-0.497588,0.168579,-0.007086,-0.001463,-0.023898,0.061535,BENIGN
2522359,-1.996934,-0.035517,0.176545,-0.675988,-1.120314,-0.864578,-0.068002,-0.609914,0.831666,-0.306117,...,0.067943,0.023721,-0.048079,-0.146745,0.127080,-0.004200,-0.001254,-0.005818,0.010076,BENIGN
2522360,-1.978716,-0.034866,0.153766,-0.752165,-1.326923,-1.081898,-0.083984,-0.682888,1.089632,-0.364274,...,0.159911,0.148349,0.032388,-0.356500,0.246884,-0.005660,-0.002753,-0.003974,0.007683,BENIGN


In [28]:
# For cross validation
from sklearn.model_selection import cross_val_score
# Creating a balanced dataset for Binary Classification
normal_traffic = new_data.loc[new_data['Attack Type'] == 'BENIGN']
intrusions = new_data.loc[new_data['Attack Type'] != 'BENIGN']

normal_traffic = normal_traffic.sample(n = len(intrusions), replace = False)

ids_data = pd.concat([intrusions, normal_traffic])
ids_data['Attack Type'] = np.where((ids_data['Attack Type'] == 'BENIGN'), 0, 1)
bc_data = ids_data.sample(n = 15000)

print(bc_data['Attack Type'].value_counts())

0    7502
1    7498
Name: Attack Type, dtype: int64


In [29]:
# Splitting the data into features (X) and target (y)
from sklearn.model_selection import train_test_split

X_bc = bc_data.drop('Attack Type', axis = 1)
y_bc = bc_data['Attack Type']

X_train_bc, X_test_bc, y_train_bc, y_test_bc = train_test_split(X_bc, y_bc, test_size = 0.25, random_state = 0)

In [30]:
new_data['Attack Type'].value_counts()

BENIGN                        2096484
DoS Hulk                       172849
DDoS                           128016
PortScan                        90819
DoS GoldenEye                   10286
FTP-Patator                      5933
DoS slowloris                    5385
DoS Slowhttptest                 5228
SSH-Patator                      3219
Bot                              1953
Web Attack � Brute Force         1470
Web Attack � XSS                  652
Infiltration                       36
Web Attack � Sql Injection         21
Heartbleed                         11
Name: Attack Type, dtype: int64

In [32]:
class_counts = new_data['Attack Type'].value_counts()
selected_classes = class_counts[class_counts > 1950]
class_names = selected_classes.index
selected = new_data[new_data['Attack Type'].isin(class_names)]

dfs = []
for name in class_names:
    df = selected[selected['Attack Type'] == name]
    if len(df) > 5000:
        df = df.sample(n=5000, random_state=0)
    # Otherwise, keep all rows (do not sample more than available)
    dfs.append(df)

df = pd.concat(dfs, ignore_index = True)
df['Attack Type'].value_counts()

BENIGN              5000
DoS Hulk            5000
DDoS                5000
PortScan            5000
DoS GoldenEye       5000
FTP-Patator         5000
DoS slowloris       5000
DoS Slowhttptest    5000
SSH-Patator         3219
Bot                 1953
Name: Attack Type, dtype: int64

In [33]:
from imblearn.over_sampling import SMOTE

X = df.drop('Attack Type', axis=1)
y = df['Attack Type']

smote = SMOTE(sampling_strategy='auto', random_state=0)
X_upsampled, y_upsampled = smote.fit_resample(X, y)

blnc_data = pd.DataFrame(X_upsampled)
blnc_data['Attack Type'] = y_upsampled
blnc_data = blnc_data.sample(frac=1)

blnc_data['Attack Type'].value_counts()

PortScan            5000
FTP-Patator         5000
SSH-Patator         5000
DoS GoldenEye       5000
DoS Hulk            5000
Bot                 5000
DoS slowloris       5000
DDoS                5000
BENIGN              5000
DoS Slowhttptest    5000
Name: Attack Type, dtype: int64

In [34]:
features = blnc_data.drop('Attack Type', axis = 1)
labels = blnc_data['Attack Type']

X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size = 0.25, random_state = 0)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
#n_estimators=10
rf1 = RandomForestClassifier(
    n_estimators=10, 
    max_depth=6, 
    max_features=None, 
    random_state=0
)
rf1.fit(X_train, y_train)

# Cross-validation scores (these are accuracies per fold)
cv_rf1 = cross_val_score(rf1, X_train, y_train, cv=5)

# Accuracy on training and test set
train_acc = accuracy_score(y_train, rf1.predict(X_train))
test_acc = accuracy_score(y_test, rf1.predict(X_test))

print("Random Forest Model 1")
print(f"Cross-validation scores: {', '.join(map(lambda x: f'{x:.2f}', cv_rf1))}")
print(f"Mean cross-validation score: {cv_rf1.mean():.2f}")
print(f"Training Accuracy: {train_acc:.2f}")
print(f"Test Accuracy: {test_acc:.2f}")


Random Forest Model 1
Cross-validation scores: 0.94, 0.94, 0.93, 0.94, 0.92
Mean cross-validation score: 0.93
Training Accuracy: 0.94
Test Accuracy: 0.93


In [ ]:

#n_estimators=15
rf2 = RandomForestClassifier(
    n_estimators=15, 
    max_depth=6, 
    max_features=None, 
    random_state=0
)
rf2.fit(X_train, y_train)

# Cross-validation scores
cv_rf2 = cross_val_score(rf2, X_train, y_train, cv=5)

# Accuracy on training and test set
train_acc2 = accuracy_score(y_train, rf2.predict(X_train))
test_acc2 = accuracy_score(y_test, rf2.predict(X_test))

print("\nRandom Forest Model 2")
print(f"Cross-validation scores: {', '.join(map(lambda x: f'{x:.2f}', cv_rf2))}")
print(f"Mean cross-validation score: {cv_rf2.mean():.2f}")
print(f"Training Accuracy: {train_acc2:.2f}")
print(f"Test Accuracy: {test_acc2:.2f}")



Random Forest Model 2
Cross-validation scores: 0.94, 0.94, 0.94, 0.94, 0.92
Mean cross-validation score: 0.94
Training Accuracy: 0.94
Test Accuracy: 0.93


In [ ]:
from sklearn.tree import DecisionTreeClassifier
#max_depth = 6

from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score


dt1 = DecisionTreeClassifier(max_depth=6, random_state=0)
dt1.fit(X_train, y_train)

# Cross-validation scores
cv_dt1 = cross_val_score(dt1, X_train, y_train, cv=5)

# Training and Test accuracy
train_acc = accuracy_score(y_train, dt1.predict(X_train))
test_acc = accuracy_score(y_test, dt1.predict(X_test))

print("Decision Tree Model 1")
print(f"Cross-validation scores: {', '.join(map(lambda x: f'{x:.2f}', cv_dt1))}")
print(f"Mean cross-validation score: {cv_dt1.mean():.2f}")
print(f"Training Accuracy: {train_acc:.2f}")
print(f"Test Accuracy: {test_acc:.2f}")


Decision Tree Model 1

Cross-validation scores: 0.9216, 0.9288, 0.9181333333333334, 0.9314666666666667, 0.9210666666666667

Mean cross-validation score: 0.92


In [ ]:
#max_depth = 8

dt2 = DecisionTreeClassifier(max_depth=8)
dt2.fit(X_train, y_train)

# Cross-validation scores
cv_dt2 = cross_val_score(dt2, X_train, y_train, cv=5)

print('Decision Tree Model 2')
print(f'\nCross-validation scores:', ', '.join(map(lambda x: f'{x:.2f}', cv_dt2)))
print(f'\nMean cross-validation score: {cv_dt2.mean():.2f}')

# Training & Test accuracy
train_acc_dt2 = accuracy_score(y_train, dt2.predict(X_train))
test_acc_dt2 = accuracy_score(y_test, dt2.predict(X_test))

print(f"Training Accuracy: {train_acc_dt2:.2f}")
print(f"Test Accuracy: {test_acc_dt2:.2f}")


Decision Tree Model 2

Cross-validation scores: 0.97, 0.98, 0.97, 0.97, 0.97

Mean cross-validation score: 0.97
Training Accuracy: 0.98
Test Accuracy: 0.97
